# Fundamentos de Apache Spark: RDDs


En este notebook trabajaremos con los RDDs que forma parte del Spark Core.La implementación de Spark Core es un **RDD (Resilient Distributed Dataset)** que es una colección de datos distribuidos en diferentes nodos del clúster que se procesan en paralelo.

Utilizaremos la API de PySpark, pero los conceptos aplican por igual a todas las APIs (Scala, R, etc)

### Inicialización de Spark en Notebooks

In [32]:
#!conda install -c conda-forge findspark

In [33]:
#import findspark
#findspark.init()
#import pandas as pd

import pyspark
from pyspark.sql import SparkSession

### Crear el SparkSession y el SparkContext

El "SparkSession" es el punto de entrada a Apache Spark

In [34]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_training')\
        .getOrCreate()

# SparkSession <-- Punto de entrada a Apache Spark, crea internamente el "SparkConfig" y el "SparkContext"
# .master("local[*]") <-- si estamos ejecutando este Notebook en un cluster, debemos indicar que estamos en el nodo maestro ".master()" 
# "local[*]" <-- indica usar todos los nucleos de nuestra computadora local, si queremos usar 2 nucleos deberiamos poner "local[2]"
#                el número de nucleos que se especifique será el tamaño en que se partirán los RDDs
# appName() <-- Sirve para especificar el nombre de la aplicación Spark
# .getOrCreate() <-- Crea un SparkSession ya existente o crea uno nuevo con las configuraciones indicadas

In [35]:
# detenemos la "SparkSession":
spark.stop()

In [36]:
# Creamos una SparkSession con las configuraciones por default:
spark = SparkSession.builder.getOrCreate()

spark

In [37]:
# Iniciamos el SparkContext
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=PySpark_training>

### Crear un RDD de una lista

In [38]:
# Creamos una lista de números:
num = list( range(20) )
print( num )

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [39]:
# Creamos el RDD usando "parallelize"
# esto permite particinar los datos entre todos los nodos del clúster
num_rdd = sc.parallelize(num)
num_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

A continuación aplicamos la acción "collect()" para coleccionar la data en el nodo maestro:

In [40]:
# Coleccionamos los datos paralelizados en el nodo maestro:
num_rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

# Transformaciones
* Como sabemos, las Transformaciones son de naturaleza perezosa y no se ejecutarán hasta que se ejecute una Acción sobre ellas.
* Intentemos comprender las distintas transformaciones disponibles.

### map
* Esto mapeará su entrada a alguna salida basada en la función especificada en la función 

In [41]:
double_rdd = num_rdd.map(lambda x : x * 2)
double_rdd.collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]

### filtro
* Para filtrar los datos en función de una determinada condición. Intentemos encontrar los números pares de num_rdd.

In [42]:
even_rdd = num_rdd.filter(lambda x : x % 2 == 0)
even_rdd.collect()

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

### flatMap
* Esta función es muy similar a map, pero puede devolver múltiples elementos para cada entrada en el RDD dado.

In [43]:
num_rdd.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [44]:
flat_rdd = num_rdd.flatMap(lambda x : range(0,x) )

# colecciona los siguientes elementos en un sólo RDD plano "flat":
# 0 -> [0]
# 1 -> [0]
# 2 -> [0,1]
# 3 -> [0,1,2]
# 4 -> [0,1,2,3]
# 5 -> [0,1,2,3,4], ...

print( flat_rdd.collect() )

[0, 0, 1, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


### distinct
* Esto devolverá elementos distintos de un RDD, es decir los valores representativos de un RDD

In [45]:
# Creamos un RDD:
rdd1 = sc.parallelize( [10, 11, 2, 10, 11, 12, 11,2,2] )

In [46]:
dist_rdd = rdd1.distinct()
dist_rdd.collect()

[10, 2, 11, 12]

### reduceByKey
* Esta función reduce los pares de valores clave en función de las claves y una función determinada dentro de reduceByKey
* Reduce la cantidad de elementos que hay en una lista de tuplas, [(’llave_1’,valor_1) , (’llave_2’,valor_2),...,(’llave_n’,valor_n) ], de acuerdo a una función f que se aplica a los valores.

In [47]:
# Creamos una lista de tuplas ("clave",valor)
pairs = [ ("a", 5), ("b", 7), ("c", 2), ("a", 3), ("b", 1), ("c", 4) , ('a', 2),]

pairs

[('a', 5), ('b', 7), ('c', 2), ('a', 3), ('b', 1), ('c', 4), ('a', 2)]

In [48]:
# Pasamos la tupla de lista a un RDD:
pair_rdd = sc.parallelize(pairs)

# Coleccionamos valores:
pair_rdd.collect()

[('a', 5), ('b', 7), ('c', 2), ('a', 3), ('b', 1), ('c', 4), ('a', 2)]

In [49]:
def f(x,y):
    return x+y

In [50]:
# Aplicamos reduceByKey de acuerdo a la función "f":
output = pair_rdd.reduceByKey(lambda val_1, val_2 : f(val_1, val_2))

result = output.collect()
result

[('a', 10), ('b', 8), ('c', 6)]

### groupByKey
* Esta función es otra función ByKey que puede operar en un par (clave, valor) RDD pero esto solo agrupará los valores basados en las claves. En otras palabras, esto solo realizará el primer paso de reduceByKey.

In [51]:
grp_out = pair_rdd.groupByKey()
grp_out.collect()

[('a', <pyspark.resultiterable.ResultIterable at 0x7f2220143250>),
 ('b', <pyspark.resultiterable.ResultIterable at 0x7f2207f262b0>),
 ('c', <pyspark.resultiterable.ResultIterable at 0x7f2207f262e0>)]

### sortByKey
* Esta función realizará la clasificación en un par (clave, valor) RDD basado en las claves. De forma predeterminada, la clasificación se realizará en orden ascendente.
* Ordena ascendentemente los elementos que hay en una lista de tuplas, [(’llave_1’,valor_1) , (’llave_2’,valor_2),...,(’llave_n’,valor_n) ], de acuerdo a los valores de cada tupla.

In [52]:
pairs = [ ("a", 5), ("d", 7), ("c", 2), ("b", 3), ("c", 5) ]
raw_rdd = sc.parallelize(pairs)

sortkey_rdd = raw_rdd.sortByKey(ascending=True)
result = sortkey_rdd.collect()
result

# Para clasificar en orden descendente, pase  “ascending=False”.

[('a', 5), ('b', 3), ('c', 2), ('c', 5), ('d', 7)]

### Ordenar por
* sortBy es una función más generalizada para ordenar.
*  Ordena ascendentemente los elementos que hay en una lista de tuplas con más de 2 elementos. El ordenamiento lo hace con respecto a los valores de la “n-ésima” componente de cada tupla.

In [53]:
# Creamos una lista de tuplas:
tuplas_general = [ ("a", 5, 10), ("d", 7, 12), ("c", 2, 11), ("b", 3, 9) ]

# Creamos el RDD:
raw_rdd = sc.parallelize(tuplas_general)

# Ordenamos de acuerdo al tercer elemento de las tuplas:
sort_out = raw_rdd.sortBy(lambda x : x[2])
result = sort_out.collect()
result


[('b', 3, 9), ('a', 5, 10), ('c', 2, 11), ('d', 7, 12)]

### countByKey().items()
* Permite extraer el número de véces en que aparece una tupla con un mismo 'key'

In [54]:
data = [ ('a',6) , ('b',3), ('c',1), ('b',2), ('a',1), ('b',1) ]

data_rdd = sc.parallelize(data)

data_rdd.countByKey().items()

dict_items([('a', 2), ('b', 3), ('c', 1)])

### countByValue().items()
* Permite extraer el número de véces en que aparece cada valor en un RDD, sirve para generar distribuciones de frecuencia

In [55]:
data = ['Python' , 'Python' , 'Scala' , 'Python', 'R', 'Scala', 'R', 'Scala', 'Python', 'Python']

# Creamos el RDD:
data_rdd = sc.parallelize(data)

data_rdd.countByValue().items() # Regresa el conteo de valores por cada item 

dict_items([('Python', 5), ('Scala', 3), ('R', 2)])

# Acciones

* Las acciones son operaciones en RDD que se ejecutan inmediatamente. Mientras que las transformaciones devuelven otro RDD, las acciones devuelven estructuras de datos nativas 

### count
* Esto contará el número de elementos en el RDD dado.

In [56]:
num_rdd = sc.parallelize([10,2,3,4,2])
num_rdd.count()

5

### first
* Esto devolverá el primer elemento del RDD dado.

In [57]:
num_rdd.first()

10

### Collect
* Esto devolverá todos los elementos para el RDD dado.


In [58]:
num_rdd.collect()

[10, 2, 3, 4, 2]

**No debemos utilizar la operación de collect mientras trabajamos con grandes conjuntos de datos**. Porque devolverá todos los datos que se distribuyen entre los diferentes trabajadores dl clúster a un controlador. Todos los datos viajarán a través de la red del trabajador al conductor y también el conductor necesitaría almacenar todos los datos. Esto obstaculizará el rendimiento de su aplicación.

### Take
* Esto devolverá los primeros "n" elementos del RDD

In [59]:
n=3
num_rdd.take(n)

[10, 2, 3]

## Apagamos la SparkSession

In [60]:
sc.stop()

Más transformaciones y acciones se pueden consultar [aquí](https://spark.apache.org/docs/latest/rdd-programming-guide.html)